## Summarizing a document

First, we will load the document and extract the text

In [3]:
import pdfplumber

with pdfplumber.open("cvm_full.pdf") as pdf:
    pdf_text = ""
    for page in pdf.pages:
        pdf_text += str(page.extract_text())

pdf_text

'COMISSÃO DE VALORES MOBILIÁRIOS\nRua Sete de Setembro, 111/2-5º e 23-34º Andares, Centro, Rio de Janeiro/RJ – CEP: 20050-901 – Brasil - Tel.: (21) 3554-8686\nRua Cincinato Braga, 340/2º, 3º e 4º Andares, Bela Vista, São Paulo/ SP – CEP: 01333-010 – Brasil - Tel.: (11) 2146-2000\nSCN Q.02 – Bl. A – Ed. Corporate Financial Center, S.404/4º Andar, Brasília/DF – CEP: 70712-900 – Brasil -Tel.: (61) 3327-2030/2031\nwww.cvm.gov.br\nRESOLUÇÃO CVM Nº 175, DE 23 DE DEZEMBRO DE 2022, COM AS ALTERAÇÕES INTRODUZIDAS\nPELAS RESOLUÇÕES CVM Nº 181/23, 184/23 E 187/23.\nDispõe sobre a constituição, o funcionamento e a\ndivulgação de informações dos fundos de\ninvestimento, bem como sobre a prestação de\nserviços para os fundos, e revoga as normas que\nespecifica.\nSUMÁRIO\nRESOLUÇÃO CVM Nº 175, DE 23 DE DEZEMBRO DE 2022, COM AS ALTERAÇÕES INTRODUZIDAS PELAs\nRESOLUÇÕES CVM Nº 181/23, 184/23 e 187/23......................................................................................... 1\nSUMÁRIO ...

In [5]:
len(pdf_text)

804000

Next, we need to count the amount of tokens.

To accomplish this we will use the Openai's `tiktoken`library, using the GPT-4 tokeniser.

In [6]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")
tokens = encoding.encode(pdf_text)

f'Amount of GPT-4 tokens: {len(tokens)}'

'Amount of GPT-4 tokens: 258487'

The goal is to implement a cheap and efficient solution.

Since the tokens length is very big (250k+), we can analyze three different approaches:

- `Prompt Stuffing`
- `MapReduce`
- `Refine`

## Prompt Stuffing

> The stuff documents chain ("stuff" as in "to stuff" or "to fill") is the most straightforward of the document chains. It takes a list of documents, inserts them all into a prompt and passes that prompt to an LLM.

In the context of document summarization, the prompt stuffing would be simply pass all the documents text as context in a single prompt, for example:
```py
"""
Summarize the document:

{document_text}
""" 
```

Pros:

- Efficient (Single LLM call)
- No context loss

Limitations:

- `Context window`: The context window is the amount of tokens a LLM is capable to receive as input. The most part of the models cannot receive 250k+ tokens.
- Probably expensive, due to the need of using a more robust model.